# Import

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

# Hyperparameter

In [2]:
config = {
    "learning_rate": 5e-5,
    "epoch": 10,
    "batch_size": 128
}

CFG = SimpleNamespace(**config)

# Load Data

In [3]:
#augmented, 500개로 맞춤
#augmented2, 1000개로 맞춤
#augmented_combined, 동일 카테고리별로 5개씩 묶고, 합친 갯수가 200개보다 적으면 200개까지 증강
#augmented_combined3, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined4, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined5, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#그리고 지역은 5000개까지 줄임
#augmented_combined6, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 7000개까지 줄임
#augmented_combined8, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 5000개까지 줄임
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 3000개까지 줄임
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 2000개까지 줄임
train_df = pd.read_csv("/kaggle/input/dataset/train_augmented_combined7.csv")
test_df = pd.read_csv("/kaggle/input/dacon-dataset/test.csv")

In [4]:
train_df["분류"].value_counts()

분류
지역               7000
경제:부동산           1727
사회:사건_사고         1284
경제:반도체           1159
사회:사회일반           740
사회:교육_시험          498
정치:국회_정당          483
사회:의료_건강          475
경제:취업_창업          423
스포츠:올림픽_아시안게임     421
경제:산업_기업          356
문화:전시_공연          336
경제:자동차            320
경제:경제일반           313
사회:장애인            311
스포츠:골프            309
정치:선거             304
경제:유통             295
IT_과학:모바일         269
사회:여성             268
사회:노동_복지          224
사회:환경             198
경제:서비스_쇼핑         194
경제:무역             188
정치:행정_자치          175
국제                169
문화:방송_연예          168
경제:금융_재테크         164
스포츠:축구            164
정치:청와대            140
문화:출판             124
IT_과학:IT_과학일반     122
IT_과학:인터넷_SNS     119
문화:미술_건축          115
정치:정치일반           111
IT_과학:과학          108
문화:문화일반           107
문화:학술_문화재         101
문화:요리_여행           95
경제:자원              89
문화:종교              87
IT_과학:콘텐츠          80
사회:미디어             64
사회:날씨              62
스포츠:농구_배구          57
문화:음악  

# Load Model

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Custom Dataset

In [6]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Data Preprocessing

In [7]:
# 데이터 준비
train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.제목_키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.제목_키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.제목_키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [8]:
test_df

,ID,제목,키워드,제목_키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대...","[부고] 김태수씨 별세 외 김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바...","신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력 신규,확진,나..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전...","[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의...","용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사 용인시,구인,장애인,구직..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경...","지자체 벽 터 경기지역 산단 활성화 모색 지자체,경기,북동부,지역,산업단지,혁신단위..."
...,...,...,...,...
23400,TEST_23400,코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속,"코로나19,감염,경로,조사,32.4%,최고,일상감염,지속,기준,확진자,기준,코로나1...","코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속 코로나19,감염,경..."
23401,TEST_23401,“여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란,"여행,외식,자제,vs,모임,시민들,혼란,인천국제공항,아시아나항공,한반도,일주,비행,...","“여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란 여행,외식,자..."
23402,TEST_23402,송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹,"임야,송철호,울산,시장,배우자,용인,매입,의혹,송철호,울산,시장,배우자,경기,용인,...","송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹 임야,송철호,울산,시장,배우자,..."
23403,TEST_23403,여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사,"여직원,자궁,모형,사진,홍보용,한의사,한의원,간호조무사,동의,자궁,모형,사진,한의사...","여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사 여직원,자궁,모형..."


In [9]:
train_df["분류"].value_counts()

분류
지역               5600
경제:부동산           1382
사회:사건_사고         1027
경제:반도체            927
사회:사회일반           592
사회:교육_시험          398
정치:국회_정당          386
사회:의료_건강          380
경제:취업_창업          338
스포츠:올림픽_아시안게임     337
경제:산업_기업          285
문화:전시_공연          269
경제:자동차            256
경제:경제일반           250
사회:장애인            249
스포츠:골프            247
정치:선거             243
경제:유통             236
IT_과학:모바일         215
사회:여성             214
사회:노동_복지          179
사회:환경             158
경제:서비스_쇼핑         155
경제:무역             150
정치:행정_자치          140
문화:방송_연예          135
국제                135
스포츠:축구            131
경제:금융_재테크         131
정치:청와대            112
문화:출판              99
IT_과학:IT_과학일반      98
IT_과학:인터넷_SNS      95
문화:미술_건축           92
정치:정치일반            89
IT_과학:과학           87
문화:문화일반            86
문화:학술_문화재          81
문화:요리_여행           76
경제:자원              71
문화:종교              70
IT_과학:콘텐츠          64
사회:미디어             51
사회:날씨              50
스포츠:농구_배구          46
문화:음악  

In [10]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
# 학습
model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())
    
    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

Validating: 100%|██████████| 33/33 [00:55<00:00,  1.68s/it]


Validation F1 Score: 0.05


Validating: 100%|██████████| 33/33 [00:55<00:00,  1.69s/it]


Validation F1 Score: 0.20


Validating: 100%|██████████| 33/33 [00:55<00:00,  1.69s/it]


...

Validation F1 Score: 0.49


Validating: 100%|██████████| 33/33 [00:55<00:00,  1.69s/it]


Validation F1 Score: 0.52


Validating: 100%|██████████| 33/33 [00:55<00:00,  1.69s/it]

Validation F1 Score: 0.54


# Inference

In [12]:
model.eval()
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        # 모델을 통한 예측
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        
        # 예측 결과 저장
        test_predictions.extend(preds.cpu().tolist())

# test.csv 파일에서 'id' 값을 그대로 사용
test_df = pd.read_csv('/kaggle/input/dacon-dataset/test.csv')  # test.csv에 'id' 칼럼이 존재한다고 가정

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]



Testing: 100%|██████████| 183/183 [03:17<00:00,  1.08s/it]


In [13]:
# 예측 결과를 데이터프레임으로 저장
df_results = pd.DataFrame({
    'ID': test_df['ID'],             # test.csv의 'id' 값을 그대로 사용
    '분류': decoded_predictions  # 디코딩된 예측 라벨
})

# CSV 파일로 저장
df_results.to_csv('/kaggle/working/submission.csv', index=False)

print("예측 결과가 submission.csv 파일로 저장되었습니다.")

예측 결과가 submission.csv 파일로 저장되었습니다.


In [14]:
df=pd.read_csv('/kaggle/working/submission.csv')

In [15]:
df.head(50)

,ID,분류
0,TEST_00000,사회:의료_건강
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:정치일반
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역
5,TEST_00005,경제:반도체
6,TEST_00006,지역
7,TEST_00007,국제
8,TEST_00008,사회:사건_사고
9,TEST_00009,지역
